Using the MovieLens data, I will try to build a hybrid recommender system.

In [ ]:
pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808329 sha256=becabff1b85a15e9ce96e29427be29cbf87d762e82b622d964f4ed02deee6232
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
dataset = Dataset()

In [ ]:
dataset.fit(ratings['userId'].unique(), ratings['movieId'].unique())

In [ ]:
(interactions, weights) = dataset.build_interactions([(row['userId'], row['movieId'], row['rating']) for index, row in ratings.iterrows()])

In [ ]:
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.model_selection import train_test_split

In [ ]:
train_interactions, test_interactions = train_test_split(interactions, test_size=0.2, random_state=42)

In [ ]:
model = LightFM(loss='warp')

In [ ]:
model.fit(train_interactions, epochs=30, num_threads=2)

In [ ]:
train_precision = precision_at_k(model, train_interactions, k=5).mean()

In [ ]:
train_precision

0.592623

In [ ]:
import numpy as np

In [ ]:
def recommend(model, user_ids, item_labels, num_recommendations=5):
    n_users, n_items = interactions.shape
    for user_id in user_ids:
        scores = model.predict(user_id, np.arange(n_items))
        top_items = np.argsort(-scores)[:num_recommendations]
        print(f'Recommendations for user {user_id}:')
        for item in top_items:
            print(f'    {item_labels[item]}')

In [ ]:
item_labels = movies['title'].tolist()

In [ ]:
recommend(model, user_ids=[1, 2, 3], item_labels=item_labels)

Recommendations for user 1:
    Get Shorty (1995)
    Cat on a Hot Tin Roof (1958)
    Chamber, The (1996)
    Shaggy Dog, The (1959)
    Grace of My Heart (1996)
Recommendations for user 2:
    Free Willy 2: The Adventure Home (1995)
    Private Benjamin (1980)
    Nico Icon (1995)
    Pocahontas (1995)
    Bed of Roses (1996)
Recommendations for user 3:
    Sense and Sensibility (1995)
    Get Shorty (1995)
    Love Affair (1994)
    It Takes Two (1995)
    Waiting to Exhale (1995)
